In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import pickle
import music21
import matplotlib.pyplot as plt

from fractions import Fraction
from collections import defaultdict, Counter
from iteration_utilities import deepflatten #flatten nested lists

from music21 import midi, note, stream, instrument, meter, key
import itertools
import random

import string
import pretty_midi

import phrase_segmentation
from phrase_segmentation import *

from pathlib import Path

In [ ]:
piece_path = Path("..") / "asap-dataset" / "Schubert" / "Impromptu_op.90_D.899" / "3"
filenames = glob.glob(os.path.join(piece_path, "*annotations.txt"))
filenames = [f for f in filenames if not f.endswith("midi_score_annotations.txt")]
sig = 4
times_list, diff_list = read_diff_timings(filenames)
times = np.array(times_list)
diff = np.array(diff_list)
print(times.shape)
print(diff.shape)

In [ ]:
avg_diff = moving_average(diff, n=5)
threshold = compute_threshold(avg_diff)

plot_avg_diff(avg_diff, threshold, sig)

In [ ]:
break_idx, breaks = beat_segmentation_marco(avg_diff, threshold, times)

In [ ]:
plot_segmentation(avg_diff, threshold, break_idx, sig)
measure_counts = np.arange(break_idx.shape[0])[break_idx] / 4 + 1
print(measure_counts)
#print(f"breaks = {breaks}")
#print(f"times={times}, break_idx={break_idx}n")

In [ ]:
mid = music21.converter.parse("../asap-dataset/Schubert/Impromptu_op.90_D.899/3/Hou06M.mid")
mid.show('midi')
